In [2]:
import random
import numpy
from itertools import combinations
# from bracket import bracket
from tkinter import *
# import pandas as pd
from tabulate import tabulate
from IPython.display import clear_output
import math

In [3]:
#test set
def test(num_players):
  lis=[]
  for i in range(num_players):
    lis.append("Player " + str(i+1))
  return lis

In [27]:
#setting parameters
per_team=2
pool_groups=2
# players=["Clay","David","Jasper","Sana","Catherine","Ronald","Derek","Nil","Kusai","Jeremy","Mahdi","Sidhant"]
players=test(20)

In [28]:
big_groups=len(players)/2%pool_groups
games_per_big=math.ceil(len(players)/per_team/pool_groups)*math.ceil(len(players)/per_team/pool_groups-1)/2
small_groups=pool_groups-len(players)/per_team%pool_groups
games_per_small=math.floor(len(players)/per_team/pool_groups)*math.floor(len(players)/per_team/pool_groups-1)/2
print(big_groups*games_per_big+small_groups*games_per_small)

20.0


In [29]:
#Make random teams
if len(players)%per_team!=0:
  raise Exception("Need an even number of players.")

random.shuffle(players)
teams=[]
for i in range(int(len(players)/per_team)):
  t="(" + players[per_team*i]
  if per_team==1:
    t+=")"
  for j in range(1,per_team):
    t+=" & " + players[per_team*i + j] + ")"
  teams.append(t)
  print(t)
# teams

(Player 2 & Player 3)
(Player 11 & Player 19)
(Player 9 & Player 1)
(Player 18 & Player 5)
(Player 15 & Player 12)
(Player 20 & Player 8)
(Player 4 & Player 10)
(Player 16 & Player 6)
(Player 14 & Player 13)
(Player 17 & Player 7)


In [30]:
#Make random matchups in random groups (*if applicable)
if pool_groups*2 > len(teams):
  raise Exception("Must have at least double the amount of teams as groups.")
random.shuffle(teams)
groups=[]
for i in range(pool_groups):
    groups.append([])
for i in range(len(teams)):
    groups[i%pool_groups].append(teams[i])
matchups=[]
for g in groups:
    for m in list(combinations(g, 2)):
        matchups.append(m)
# matchups = list(combinations(teams, 2))
random.shuffle(matchups)
  
#randomize matchups
list_of_matchups=[]
for match in matchups:
  # m=random.shuffle(list(match))
  match=list(match)
  random.shuffle(match)
  mat=[match[0]]
  st=match[0]
  for j in range(1,2):
    mat.append(match[j])
    st+=" vs " + match[j]
  list_of_matchups.append(mat)
  print(st)
# list_of_matchups

(Player 14 & Player 13) vs (Player 2 & Player 3)
(Player 4 & Player 10) vs (Player 18 & Player 5)
(Player 20 & Player 8) vs (Player 18 & Player 5)
(Player 20 & Player 8) vs (Player 16 & Player 6)
(Player 2 & Player 3) vs (Player 17 & Player 7)
(Player 17 & Player 7) vs (Player 14 & Player 13)
(Player 18 & Player 5) vs (Player 9 & Player 1)
(Player 9 & Player 1) vs (Player 4 & Player 10)
(Player 16 & Player 6) vs (Player 9 & Player 1)
(Player 14 & Player 13) vs (Player 15 & Player 12)
(Player 16 & Player 6) vs (Player 18 & Player 5)
(Player 4 & Player 10) vs (Player 20 & Player 8)
(Player 11 & Player 19) vs (Player 14 & Player 13)
(Player 11 & Player 19) vs (Player 15 & Player 12)
(Player 15 & Player 12) vs (Player 17 & Player 7)
(Player 9 & Player 1) vs (Player 20 & Player 8)
(Player 11 & Player 19) vs (Player 2 & Player 3)
(Player 15 & Player 12) vs (Player 2 & Player 3)
(Player 17 & Player 7) vs (Player 11 & Player 19)
(Player 4 & Player 10) vs (Player 16 & Player 6)


In [31]:
#Setting dictionary for every team.
def set_dict(teams):
    standings = dict()

    for t in teams:
        tup={"Wins":0,"Losses":0,"Ties":0,"GF":0,"GA":0,"WF":[],"WA":[]}
        standings[t]=tup

    return standings

In [32]:
#helper functions for sorting standings
def win_percentage(t, standings):
    number_of_games=(standings[t]["Wins"]+standings[t]["Losses"]+standings[t]["Ties"])
    return (standings[t]["Wins"]+0.5*standings[t]["Ties"])/number_of_games

def win_diff(t,standings):
    return standings[t]["Wins"]-standings[t]["Losses"]

def point_diff(t,standings):
    return standings[t]["GF"]-standings[t]["GA"]
    
def tied_winperc(t1,t2,standings,tied_teams=None):
    return win_percentage(t1,standings)==win_percentage(t2,standings) and win_diff(t1,standings)==win_diff(t2,standings) and \
point_diff(t1,standings) == point_diff(t2,standings) and standings[t1]["GF"] == standings[t2]["GF"]

def tied_windiff(t1,t2,standings,tied_teams):
    wins1=sum(standings[t1]["WF"].count(x) for x in tied_teams)
    losses1=sum(standings[t1]["WA"].count(x) for x in tied_teams)
    wins_against1=wins1-losses1
    
    wins2=sum(standings[t2]["WF"].count(x) for x in tied_teams)
    losses2=sum(standings[t2]["WA"].count(x) for x in tied_teams)
    wins_against2=wins2-losses2
    return wins_against1==wins_against2

def tied_pointdiff(t1,t2,standings,tied_teams=None):
    return standings[t1]["GF"]-standings[t1]["GA"]==standings[t2]["GF"]-standings[t2]["GA"]

def tied_points(t1,t2,standings,teams=None):
    return standings[t1]["GF"]==standings[t2]["GF"]

#driver method for sorting.
def sort_standings(teams,standings):
    return sort(teams,standings,tied_winperc)

def sort(teams,standings,tied,tied_teams=None):
    # return sorted(teams,key=lambda t:(win_percentage(t,standings),win_diff(t,standings)),reverse=True)
    tied_count=1
    #Now we will settle tie breakers.
    for i in range(len(teams)-1):
        if tied(teams[i],teams[i+1],standings,teams):
            tied_count+=1
            # print(settle_tie(teams[i+2-tied_count:i+2],standings))
        else:
            if tied_count>1:
                teams[i+1-tied_count:i+1]=settle_tie(teams[i+1-tied_count:i+1],standings)
                # print(teams[i+2-tied_count:i+2])
            tied_count=1
    #account for last place tie.
    if tied_count>1:
        teams[-tied_count:]=settle_tie(teams[-tied_count:],standings)
        # print(teams[-tied_count:])
    return teams

#Keep commented prints for testing purposes.

def settle_tie(tied_teams,standings):
    # random.shuffle(tied_teams)
    # return tied_teams
    wins_against=dict()
    for t in tied_teams:
        wins=sum(standings[t]["WF"].count(x) for x in tied_teams)
        losses=sum(standings[t]["WA"].count(x) for x in tied_teams)
        wins_against[t]=wins-losses
    tied_teams=sorted(tied_teams,key=lambda t:wins_against[t],reverse=True)
    #convert values to list and remove duplicates. Length is 1 if it is all the same.
    if len(list(set(list(wins_against.values())))) == 1:
        return settle_windiff(tied_teams,standings)
    else:
        # print("worked 1")
        # print(wins_against)
        return sort(tied_teams,standings,tied_windiff)
    
def settle_windiff(tied_teams,standings):
    # random.shuffle(tied_teams)
    # return tied_teams
    goals_against=dict()
    for t in tied_teams:
        gf=standings[t]["GF"]
        ga=standings[t]["GA"]
        goals_against[t]=gf-ga
    tied_teams=sorted(tied_teams,key=lambda t:goals_against[t],reverse=True)
    if len(list(set(list(goals_against.values())))) == 1:
        return settle_pointdiff(tied_teams,standings)
    else:
        # print("worked 2")
        # print(goals_against)
        return sort(tied_teams,standings,tied_pointdiff)
    
def settle_pointdiff(tied_teams,standings):
    # random.shuffle(tied_teams)
    # return tied_teams
    goals=dict()
    for t in tied_teams:
        goals[t]=standings[t]["GF"]
    tied_teams=sorted(tied_teams,key=lambda t:goals[t],reverse=True)
    #Just do random if none of the tie breakers settle it.
    if len(list(set(list(goals.values())))) == 1:
        # print("worked 4")
        # print(tied_teams)
        random.shuffle(tied_teams)
        return tied_teams
    else:
        # print("worked 3")
        # print(tied_teams)
        return sort(tied_teams,standings,tied_points)
    

In [33]:
#Setting up GUI for matchups.
root = Tk()
root.title("Entry Boxes")
root.geometry("1280x720")
# root.iconbitmap('C:\Users\18317\Documents\Foosball\test')

my_entries=[] 

def update_scores():
    clear_output(wait=True)
    standings=set_dict(teams)
    vec=[]
    for i in range(int(len(my_entries)/2)):
        # val1= my_entries[2*i].get()
        # val2= my_entries[2*i+1].get()
        # vec.append([int(val1) if val1.isnumeric() else 0,int(val2) if val2.isnumeric() else 0])
        val1=3 if random.randint(0,1)==0 else random.randint(0,2)
        val2=3 if val1!=3 else random.randint(0,2)
        vec.append([val1,val2])
    num=0
    for game in vec:
        if game[0]==game[1]:
            standings[list_of_matchups[num][0]]["Ties"]=standings[list_of_matchups[num][0]]["Ties"]+1
            standings[list_of_matchups[num][1]]["Ties"]=standings[list_of_matchups[num][1]]["Ties"]+1
            
            goals=game[0]
            standings[list_of_matchups[num][0]]["GF"]=standings[list_of_matchups[num][0]]["GF"]+goals
            standings[list_of_matchups[num][0]]["GA"]=standings[list_of_matchups[num][0]]["GA"]+goals
            standings[list_of_matchups[num][1]]["GF"]=standings[list_of_matchups[num][1]]["GF"]+goals
            standings[list_of_matchups[num][1]]["GA"]=standings[list_of_matchups[num][1]]["GA"]+goals
        else:
            ind=game.index(max(game))
            winner=list_of_matchups[num][ind]
            loser=list_of_matchups[num][(ind+1)%2]

            standings[winner]["WF"].append(loser)
            standings[winner]["Wins"]=standings[winner]["Wins"]+1
            standings[winner]["GF"]=standings[winner]["GF"]+game[ind]
            standings[winner]["GA"]=standings[winner]["GA"]+game[(ind+1)%2]

            standings[loser]["WA"].append(winner)
            standings[loser]["Losses"]=standings[loser]["Losses"]+1
            standings[loser]["GF"]=standings[loser]["GF"]+game[(ind+1)%2]
            standings[loser]["GA"]=standings[loser]["GA"]+game[ind]
        
        print(list_of_matchups[num][0] + "\t" +str(game[0]) + " - " + str(game[1]) + "\t" + list_of_matchups[num][1])
        num+=1
    col_names=["Seed","Team","Games Played","Wins","Losses","Ties","Points","Win%","Point Differential"]
    table=[]
    #sort teams first by win percentage and then win/loss differential.
    sorted_teams=sorted(teams,key=lambda t:(win_percentage(t,standings),win_diff(t,standings),point_diff(t,standings),standings[t]["GF"]),reverse=True)
    sorted_teams_test=sort_standings(sorted_teams.copy(),standings)
    seed=1
    # for t in sorted_teams:
    #     number_of_games=(standings[t]["Wins"]+standings[t]["Losses"]+standings[t]["Ties"])
    #     table.append([seed,t,number_of_games,standings[t]["Wins"],standings[t]["Losses"],standings[t]["Ties"],standings[t]["GF"],
    #                   0 if number_of_games==0 else round((standings[t]["Wins"]+0.5*standings[t]["Ties"])/number_of_games,3),
    #                   standings[t]["GF"]-standings[t]["GA"]])
    #     seed+=1
    # table=sorted(table,key=lambda x:x[0])
    # print(tabulate(table,headers=col_names, tablefmt="fancy_grid"))
    
    #test
    table=[]
    for t in sorted_teams_test:
        number_of_games=(standings[t]["Wins"]+standings[t]["Losses"]+standings[t]["Ties"])
        table.append([seed,t,number_of_games,standings[t]["Wins"],standings[t]["Losses"],standings[t]["Ties"],standings[t]["GF"],
                      0 if number_of_games==0 else round((standings[t]["Wins"]+0.5*standings[t]["Ties"])/number_of_games,3),
                      standings[t]["GF"]-standings[t]["GA"]])
        seed+=1
    table=sorted(table,key=lambda x:x[0])
    print(tabulate(table,headers=col_names, tablefmt="fancy_grid"))
    
# scrollbar = Scrollbar(root)
# scrollbar.pack( side = RIGHT, fill = Y )
    
for x in range(len(list_of_matchups)):
    for y in range(2):
        labelNum = Label(root, text=list_of_matchups[x][y]).grid(row=2*x, column=2*y)  
        my_entry=Entry(root,justify='center')
        my_entry.grid(row=2*x+1, column=2*y)
        my_entries.append(my_entry)
    
my_button = Button(root, text="Save",command=update_scores,height=1,width=10)
my_button.grid(row=1,column=y+2,padx=20)

root.bind('<Return>',lambda event:update_scores())

# my_label = Label(root, text='')
# my_label.grid(row=x+2,column=0,pady=20)

root.mainloop()


(Player 14 & Player 13)	1 - 3	(Player 2 & Player 3)
(Player 4 & Player 10)	3 - 2	(Player 18 & Player 5)
(Player 20 & Player 8)	3 - 0	(Player 18 & Player 5)
(Player 20 & Player 8)	3 - 1	(Player 16 & Player 6)
(Player 2 & Player 3)	3 - 1	(Player 17 & Player 7)
(Player 17 & Player 7)	0 - 3	(Player 14 & Player 13)
(Player 18 & Player 5)	3 - 0	(Player 9 & Player 1)
(Player 9 & Player 1)	3 - 2	(Player 4 & Player 10)
(Player 16 & Player 6)	3 - 2	(Player 9 & Player 1)
(Player 14 & Player 13)	2 - 3	(Player 15 & Player 12)
(Player 16 & Player 6)	2 - 3	(Player 18 & Player 5)
(Player 4 & Player 10)	1 - 3	(Player 20 & Player 8)
(Player 11 & Player 19)	3 - 1	(Player 14 & Player 13)
(Player 11 & Player 19)	2 - 3	(Player 15 & Player 12)
(Player 15 & Player 12)	2 - 3	(Player 17 & Player 7)
(Player 9 & Player 1)	0 - 3	(Player 20 & Player 8)
(Player 11 & Player 19)	3 - 0	(Player 2 & Player 3)
(Player 15 & Player 12)	3 - 1	(Player 2 & Player 3)
(Player 17 & Player 7)	0 - 3	(Player 11 & Player 19)
(Player 

In [86]:
# root = Tk()
# root.geometry("300x300")
# root.title(" Q&A ")
 
# def Take_input():
#     INPUT = inputtxt.get("1.0", "end-1c")
#     print(INPUT)
#     if(INPUT == "120"):
#         Output.insert(END, 'Correct')
#     else:
#         Output.insert(END, "Wrong answer")
     
# l = Label(text = "What is 24 * 5 ? ")
# inputtxt = Text(root, height = 1,
#                 width = 5,
#                 bg = "light yellow")

# inputtxt2 = Text(root, height = 1,
#                 width = 5,
#                 bg = "light green")
 
# Output = Text(root, height = 5,
#               width = 25,
#               bg = "light cyan")
 
# Display = Button(root, height = 2,
#                  width = 20,
#                  text ="Show",
#                  command = lambda:Take_input())
 
# l.pack()
# inputtxt.pack()
# Display.pack()
# Output.pack()
 
# mainloop()

In [22]:
import math
import random
import tkinter
import statistics


def generate_bracket_rec(players):
    games_map = {}
    next_players = []

    assert len(players) > 0

    if len(players) == 1:
        return games_map

    for i in range(int(len(players) / 2)):
        player1_id = i
        player2_id = len(players) - 1 - i

        if players[player2_id] is None:
            next_players.append(players[player1_id])
        else:
            game_id = str(int(math.log(len(players), 2))) + "g" + str(i + 1)
            g = Game((players[player1_id], players[player2_id]), game_id)
            games_map[g.id] = g
            next_players.append((g, "winner"))
    d = {}
    d.update(games_map)
    d.update(generate_bracket_rec(next_players).items())
    return d


# Players in each game will either be string literals
# or tuples that contain a Game and "loser" or "winner"
class Game:
    def __init__(self, players, game_id=None):
        self.players = players
        self.done = False
        self.id = random.getrandbits(24) if game_id is None else game_id

    def __str__(self):
        output = self.id + ": "
        first_raw = self.players[0]
        output += (first_raw[0].get(first_raw[1]) if type(self.players[0]) is tuple else first_raw) + " vs "
        output += (self.players[1][0].get(self.players[1][1]) if type(self.players[1]) is tuple else self.players[1])
        return output

    def __repr__(self):
        output = str(self)
        output = output + "\nDone: " + \
            str(self.done) + (" | Winner: " + self.winner + " Loser: " + self.loser if self.done else "")
        return output

    def get_raw_players(self):
        return tuple(player[0].get(player[1]) if type(player) is tuple else player for player in self.players)

    def is_ready(self):
        for player in self.players:
            if type(player) is tuple and not player[0].done:
                return False
        return True

    def play_game(self, winning_player):
        # TODO: Deal with fact that this is just the is_ready method again
        for player in self.players:
            if type(player) is tuple and not player[0].done:
                print("ERROR 1: Attempting to start " + self.id + " before " + player[0].id + " is complete")
                return False

        players_raw = tuple(player[0].get(player[1]) if type(player) is tuple else player for player in self.players)
        if winning_player not in players_raw:
            print("ERROR 2: " + winning_player + " not in " + self.id)
            return False

        self.winner = winning_player
        self.loser = players_raw[0] if players_raw[1] == winning_player else players_raw[1]
        self.done = True
        return True

    def get(self, item):
        if not self.done:
            return str(item) + " of " + str(self.id)
        else:
            if item == "winner":
                return self.winner
            elif item == "loser":
                return self.loser
            else:
                return "INVALID"

# TODO: Make a more a general get function that checks if
# the type is a Tuple or a game and then evaluates


class Tournament:
    def __init__(self, players):
        self.players = players
        self.player_count = len(players)
        self.games = {}

    def __str__(self):
        output = ""
        for s in players:
            output = output + ((s + " ") if s is not None else "")
        output += "\nlog(Size): " + str(self.n_size)
        output += " | underflow: " + str(self.underflow)
        return output

    def __repr__(self):
        return self.__str__()

    def generate_bracket(self, bracket_class, randomize):
        if bracket_class != "single":
            print("Illegal bracket class")
            quit()

        self.n_size = int(math.ceil(math.log(self.player_count, 2)))
        self.underflow = int(math.pow(2, self.n_size) - self.player_count)

        if randomize:
            random.shuffle(self.players)

        for i in range(self.underflow):
            self.players.append(None)

        self.games = generate_bracket_rec(self.players)

    def play_game(self, game_id, game_winner):
        if game_id not in self.games:
            print("ERROR 3: Illegal Game ID " + game_id)
            return False

        if self.games[game_id].done:
            print("ERROR 4: Game aready complete " + game_id)
            return False

        return self.games[game_id].play_game(game_winner)


class TournamentGUI:
    def __init__(self, master, tournament):
        self.master = master
        master.title("Tournament GUI")

        self.label = tkinter.Label(master, text="Available Games: " + str([str(g) for g in t.games.values() if g.is_ready() and not g.done]))
        self.label.pack()

        self.status = tkinter.Label(master, text="Enter a Game ID and Winner.")
        self.status.pack()

        self.textBox_game = tkinter.Text(master, height=2, width=10)
        self.textBox_game.pack()

        self.textBox_winner = tkinter.Text(master, height=2, width=10)
        self.textBox_winner.pack()

        self.greet_button = tkinter.Button(
            master, text="Play Game", command=self.play_game)
        self.greet_button.pack()

    def play_game(self):
        result = t.play_game(self.textBox_game.get("1.0", 'end-1c').strip(), self.textBox_winner.get("1.0", 'end-1c').strip())
        self.label['text'] = str([str(g) for g in t.games.values() if g.is_ready() and not g.done])
        if result:
            self.textBox_game.delete('1.0', tkinter.END)
            self.textBox_winner.delete('1.0', tkinter.END)
            self.status['text'] = "Enter a Game ID and Winner."
        else:
            self.status['text'] = "Error. Re-enter game and winner."


players = ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j"]
t = Tournament(players)
t.generate_bracket("single", True)

root = tkinter.Tk()

"""
BRACKET DRAWING PRROF OF CONCEPT
"""

HEIGHT = 500  # root.winfo_screenheight()
WIDTH = 1000  # root.winfo_screenwidth()
HORIZONTAL_PADDING = 70
GAME_BOX_WIDTH_HEIGHT_RATIO = 3

_size = 3
_columns = _size * 2 + 1
_column_width = WIDTH / _columns
_game_box_width = _column_width - HORIZONTAL_PADDING
_game_box_height = _game_box_width / GAME_BOX_WIDTH_HEIGHT_RATIO


canvas = tkinter.Canvas(root, width=WIDTH, height=HEIGHT)
canvas.pack()

for i in range(_columns):
    if i - _size < 0:
        side = "LEFT"
    elif i - _size > 0:
        side = "RIGHT"
    else:
        side = "CENTER"
    games = 2 ** abs(i - _size)
    x_center = _column_width * (i + 0.5)
    y_size = HEIGHT / games
    for j in range(games):
        y_center = y_size * (j + 0.5)
        canvas.create_rectangle(x_center - _game_box_width / 2, y_center - _game_box_height / 2, x_center + _game_box_width / 2, y_center + _game_box_height / 2)
        canvas.create_text(x_center,y_center,text="Clay\nDavid",font=("toledo",int(_game_box_width/10)))
        if i != _columns - 1:
            canvas.create_line(x_center + _game_box_width / 2, y_center, x_center + _game_box_width / 2 + HORIZONTAL_PADDING / 2, y_center)
        if i != 0:
            canvas.create_line(x_center - _game_box_width / 2, y_center, x_center - _game_box_width / 2 - HORIZONTAL_PADDING / 2, y_center)

        if j % 2 == 1:
            if side == "LEFT":
                canvas.create_line(x_center + _game_box_width / 2 + HORIZONTAL_PADDING / 2, y_center, x_center + _game_box_width / 2 + HORIZONTAL_PADDING / 2, y_center - y_size)
            if side == "RIGHT":
                canvas.create_line(x_center - _game_box_width / 2 - HORIZONTAL_PADDING / 2, y_center, x_center - _game_box_width / 2 - HORIZONTAL_PADDING / 2, y_center - y_size)
                

tkinter.mainloop()